In [ ]:
# Imports
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
# Load the iris dataset.
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
# Split the dataset into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the data.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Create the Keras model.
def create_model(num_inputs, num_outputs):
    model = Sequential(
        [
            Dense(10, input_dim=num_inputs, activation='relu'),
            Dense(num_outputs, activation='softmax')
        ]
    )
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Define the fitness function.
def fitness_func(solution, solution_idx):
    model.set_weights(solution)
    y_pred = model.predict(X_train)
    mse = np.mean((y_train - y_pred) ** 2)
    return 1.0 / (mse + 1e-6)

In [ ]:
# Define the parameters.
num_inputs = X_train.shape[1]
num_outputs = len(np.unique(y_train))
sol_per_pop = 10
num_generations = 50
mutation_rate = 0.1

In [ ]:
# Define the callback function.
def on_generation(ga_instance):
    print(f'Generation: {ga_instance.generations_completed}')
    print(f'Fitness: {ga_instance.best_solution()[1]}')

In [ ]:
# Create the model.
model = create_model(num_inputs, num_outputs)
initial_weights = model.get_weights()
keras_ga = pygad.kerasga.KerasGA(
    model=model,
    num_solutions=sol_per_pop,
    num_parents_mating=sol_per_pop // 2,
    initial_population=initial_weights,
    mutation_percent_genes=mutation_rate * 100,
    fitness_func=fitness_func,
    on_generation=on_generation
)

In [ ]:
# Run the genetic algorithm.
keras_ga.run(num_generations=num_generations)

In [ ]:
# Evaluate the final model on the testing set.
best_solution = keras_ga.best_solution()
model.set_weights(best_solution)
y_pred = model.predict(X_test)
accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
print(f'Test Accuracy: {accuracy:.2f}')

In [ ]:
# Plot the fitness history.
pygad.plotting.plot_fitness_history(keras_ga.fitness_history)
plt.show()